In [ ]:
import numpy
import pygetm
import pygetm.domain

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Set up rectangular domain
domain = pygetm.domain.Domain.create_cartesian(numpy.linspace(0., 50000, 100), numpy.linspace(0., 30000, 30), 1, periodic_x=True, lat=0., H=50)
sim = pygetm.Simulation(domain, runtype=1, advection_scheme=1)

# Idealized surface forcing
tausx, tausx_ = domain.T.array(fill=0.)
tausx[...] = 0.01
tausy_ = numpy.zeros_like(tausx_)
sp, sp_ = domain.T.array(fill=0.)

# Time
timestep = 10.
ntime = int(3600. // timestep)

# Set up velocity plot
nsample = 4
fig, ax = matplotlib.pyplot.subplots()
pc = ax.pcolormesh(domain.T.xi, domain.T.yi, domain.T.z, vmin=-0.001, vmax=0.001)
cb = fig.colorbar(pc)
cb.set_label('elevation (m)')
Q = ax.quiver(domain.T.x[::nsample, ::nsample], domain.T.y[::nsample, ::nsample], sim.momentum.U[::nsample, ::nsample], sim.momentum.V[::nsample, ::nsample], scale=.3)
title = ax.set_title('time: 0 s')

In [ ]:
dist_U = domain.distribute(sim.momentum.U_)
dist_V = domain.distribute(sim.momentum.V_)
dist_U.update_halos()
dist_V.update_halos()
plotting_interval = 5
times = timestep * numpy.arange(ntime)
Us = numpy.empty_like(times)
for istep, time in enumerate(times):
    Us[istep] = sim.momentum.U.mean()
    
    sim.pressure.surface(domain.T.z_, sp_)
    sim.momentum.uv_momentum_2d(timestep, tausx_, tausy_, sim.pressure.dpdx_, sim.pressure.dpdy_)
    dist_U.update_halos()
    dist_V.update_halos()
    sim.sealevel.update(timestep, sim.momentum.U_, sim.momentum.V_)
    sim.update_depth()

    # Update velocity plot
    if istep % plotting_interval == 0:
        Q.set_UVC(sim.momentum.U[::nsample, ::nsample], sim.momentum.V[::nsample, ::nsample])
        title.set_text('time: %s s' % time)
        pc.set_array(domain.T.z.ravel())
        fig.canvas.draw()    

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
ax.plot(times, Us)
ax.set_title('mean U')
ax.grid()

# Let's try this with an island in the center of the domain

In [ ]:
# Set up rectangular domain with and circular island in the center
domain = pygetm.domain.Domain.create_cartesian(numpy.linspace(0., 50000, 100), numpy.linspace(0., 30000, 30), 1, periodic_x=True, lat=0., H=50)
distance_from_center = numpy.sqrt((domain.x - 25000)**2 + (domain.y - 15000)**2)
domain.mask[distance_from_center < 5000] = 0
sim = pygetm.Simulation(domain, runtype=1, advection_scheme=1)

# Idealized surface forcing
tausx, tausx_ = domain.T.array(fill=0.)
tausx[...] = 0.01
tausy_ = numpy.zeros_like(tausx_)
sp, sp_ = domain.T.array(fill=0.)

# Time
timestep = 5.
ntime = int(3600. // timestep)

# Set up velocity plot
nsample = 4
fig, ax = matplotlib.pyplot.subplots()
masked_z = numpy.ma.array(domain.T.z, mask=domain.T.mask==0)
pc = ax.pcolormesh(domain.T.xi, domain.T.yi, masked_z, vmin=-0.001, vmax=0.001)
cb = fig.colorbar(pc)
cb.set_label('elevation (m)')
Q = ax.quiver(domain.T.x[::nsample, ::nsample], domain.T.y[::nsample, ::nsample], sim.momentum.U[::nsample, ::nsample], sim.momentum.V[::nsample, ::nsample], scale=.3)
title = ax.set_title('time: 0 s')

In [ ]:
dist_U = domain.distribute(sim.momentum.U_)
dist_V = domain.distribute(sim.momentum.V_)
dist_U.update_halos()
dist_V.update_halos()
plotting_interval = 5
times = timestep * numpy.arange(ntime)
Us = numpy.empty_like(times)
for istep, time in enumerate(times):
    Us[istep] = sim.momentum.U.mean()
    
    sim.pressure.surface(domain.T.z_, sp_)
    sim.momentum.uv_momentum_2d(timestep, tausx_, tausy_, sim.pressure.dpdx_, sim.pressure.dpdy_)
    dist_U.update_halos()
    dist_V.update_halos()
    sim.sealevel.update(timestep, sim.momentum.U_, sim.momentum.V_)
    sim.update_depth()

    # Update velocity plot
    if istep % plotting_interval == 0:
        Q.set_UVC(sim.momentum.U[::nsample, ::nsample], sim.momentum.V[::nsample, ::nsample])
        title.set_text('time: %s s' % time)
        pc.set_array(masked_z.ravel())
        fig.canvas.draw()    